# 0. Imports, libraries and rusable functions

In [3]:
# Standard Library Imports
import ast
import copy
import csv
import json
import math
import os
import re
import time
import warnings
import logging
import random
import collections
from collections import Counter, defaultdict
from typing import List, Tuple, Optional
from IPython.display import HTML, display
import math
import time
from unidecode import unidecode
import string
import multiprocessing as mp



# Data Handling Libraries
import numpy as np
import pandas as pd
import csv
from torch.utils.data import random_split
import datasets
from datasets import ClassLabel, Sequence, Dataset, DatasetDict, load_dataset, load_metric, concatenate_datasets, load_from_disk


# Data Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
# import scikitplot as skplt  # Uncomment if scikit-plot is installed and needed

# Machine Learning: Model Preparation
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, f1_score
from sklearn.model_selection import cross_val_score, cross_validate, KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler

# Machine Learning: Models and Frameworks
import tensorflow as tf
import torch
from torch.utils.data import DataLoader
import evaluate
import xgboost
import wandb
from xgboost import plot_importance  # Uncomment if xgboost importance plot is required


# NLP and Transformers
import spacy
import transformers
from transformers import (AdamW, AutoModelForSequenceClassification, AutoModelForQuestionAnswering, AutoModelForMultipleChoice,
                          AutoTokenizer, CamembertForSequenceClassification, DistilBertConfig,
                          DistilBertForSequenceClassification, DistilBertModel, EarlyStoppingCallback,
                          get_linear_schedule_with_warmup, RobertaForSequenceClassification, EvalPrediction,
                          Trainer, TrainerCallback, TrainingArguments, XLMRobertaForSequenceClassification,
                         DefaultDataCollator, BertForQuestionAnswering, DataCollatorWithPadding, PreTrainedTokenizerFast,
                         default_data_collator, is_torch_xla_available, pipeline)
from transformers.trainer_utils import PredictionOutput, speed_metrics

# Experiment Tracking and Optimization Utilities
import optuna
from optuna.trial import TrialState
# import wandb  # Uncomment if using Weights & Biases for experiment tracking

# Progress Bar Utilities
from tqdm.auto import tqdm


In [4]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

GPU: NVIDIA GeForce RTX 4070 Ti SUPER is available.
cuda


# 1. Global Variables

In [6]:
## Arguments and global vriables
dataset_name="LogiQA"
pretrained_model_name = "microsoft/deberta-v3-base"
normalized_model_name = pretrained_model_name.replace("/", "-")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
data_collator = DefaultDataCollator()
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
pad_on_right = right_padding = tokenizer.padding_side == 'right'
global_counter = 0
traing_answer_mismatches = []
logger = logging.getLogger(__name__)

C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


# 2. Prepare the Dataset 

In [8]:
# Load the combined dataset
combined_dataset = load_from_disk('cleaned_dataset')

combined_dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Question', 'Options', 'Label_Text', 'Label', 'Type', 'Source Dataset'],
        num_rows: 1072514
    })
    validation: Dataset({
        features: ['Context', 'Question', 'Options', 'Label_Text', 'Label', 'Type', 'Source Dataset'],
        num_rows: 118521
    })
    test: Dataset({
        features: ['Context', 'Question', 'Options', 'Label_Text', 'Label', 'Type', 'Source Dataset'],
        num_rows: 200566
    })
})

In [9]:
# Filter the dataset to only include LogiQA 2.0 data
logiqa_train = combined_dataset['train'].filter(lambda x: x['Source Dataset'] == 'LogiQA 2.0')
logiqa_val = combined_dataset['validation'].filter(lambda x: x['Source Dataset'] == 'LogiQA 2.0')
logiqa_test = combined_dataset['test'].filter(lambda x: x['Source Dataset'] == 'LogiQA 2.0')


In [10]:
# Preprocessing function for multiple-choice tasks
def mcqa_preprocess_function(examples):
    num_choices = num_choices = len(examples['Options'][0])    
    first_sentences = [[context] * num_choices for context in examples['Context']]  # Repeat context for each option
    question_headers = examples['Question']
    options_list = examples['Options']
    
    second_sentences = []
    for question, options in zip(question_headers, options_list):
        # Combine question with each option
        second_sentences.append([f"{question} {option}" for option in options])
    
    # Flatten the lists
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    
    # Tokenize the inputs
    tokenized_examples = tokenizer(
        first_sentences,
        second_sentences,
        truncation=True,
        max_length=512,
        padding='max_length',
    )
    
    # Un-flatten the tokenized inputs to have shape (num_examples, num_choices, seq_length)
    tokenized_inputs = {k: [v[i:i + num_choices] for i in range(0, len(v), num_choices)] for k, v in tokenized_examples.items()}
    
    # Labels
    tokenized_inputs["labels"] = examples["Label"]
    
    return tokenized_inputs

# Apply the preprocessing function to the datasets
encoded_logiqa_train = logiqa_train.map(mcqa_preprocess_function, batched=True)
encoded_logiqa_val = logiqa_val.map(mcqa_preprocess_function, batched=True)
encoded_logiqa_test = logiqa_test.map(mcqa_preprocess_function, batched=True)

In [45]:
# Set the format of the datasets to PyTorch tensors
encoded_logiqa_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
encoded_logiqa_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
encoded_logiqa_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

def get_train_encoded():
    return encoded_logiqa_train

def get_val_encoded():
    return encoded_logiqa_val

def get_test_encoded():
    return encoded_logiqa_test

# 3. Reusable Functions

In [13]:
# Load the accuracy metric
accuracy = evaluate.load('accuracy')

# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)['accuracy']
    f1 = f1_score(labels, predictions, average='weighted')
    return {'eval_accuracy': acc, 'eval_f1': f1}

In [14]:
from transformers import TrainingArguments

def create_training_args(run_name="Default-Run", num_train_epochs=3, learning_rate=4.92e-05, batch_size=3):
    """
    Generates training arguments for training a machine learning model.

    Parameters:
    - dataset_name (str): The name of the dataset.
    - run_name (str): The name of the run, useful for logging and saving models.
    - model_name (str): The name of the model, typically including its configuration.
    - num_train_epochs (int): The number of epochs to train for.
    - learning_rate (float): The learning rate for training.
    - batch_size (int): The batch size used for training.

    Returns:
    - TrainingArguments: A configured TrainingArguments instance.
    """    
    output_dir = f"./{dataset_name}/{run_name}/{normalized_model_name}"
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        metric_for_best_model='eval_accuracy',
        greater_is_better=True,
        load_best_model_at_end=True,
        save_total_limit=3,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=1,
        warmup_steps=398,
        weight_decay=0.194,
        adam_beta1=0.837,
        adam_beta2=0.997,
        adam_epsilon=5.87e-07,
        lr_scheduler_type='cosine',
        fp16=True,  # Enable mixed-precision training
    )
    
    return training_args


In [48]:
def create_trainer(run_name="Default-Run", num_train_epochs=3, learning_rate=4.92e-05, batch_size=4):
    trainer = Trainer(
        model=model,
        args=create_training_args(run_name=run_name, num_train_epochs=num_train_epochs, learning_rate=learning_rate, batch_size=batch_size),
        train_dataset=get_train_encoded(),
        eval_dataset=get_val_encoded(),
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    
    return trainer


# 4. Fine-tuning DeBERTa on the Dataset

## 4.1 Evaluate Vanilla DeBERTa (Acc=27.75%)

In [17]:
# Load the model
model = AutoModelForMultipleChoice.from_pretrained(pretrained_model_name)

# Create the Trainer
trainer = create_trainer()

# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print(f"Test Results: {test_results}")

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: mzak071 (COMPSCI714). Use `wandb login --relogin` to force relogin


Test Results: {'eval_accuracy': 0.27753023551877787, 'eval_f1': 0.27773421919368274, 'eval_loss': 1.386296033859253, 'eval_model_preparation_time': 0.0, 'eval_runtime': 43.6793, 'eval_samples_per_second': 35.967, 'eval_steps_per_second': 35.967}


## 4.2 Fine-Tune and Evaluate Vanilla DeBERTa (Acc=23.29%)

In [19]:
# Load the model
model = AutoModelForMultipleChoice.from_pretrained(pretrained_model_name)
# Create the Trainer
trainer = create_trainer()
# Train the model
trainer.train()
# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print(f"Test Results: {test_results}")

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.391300,1.386719,0.231995,0.088533
2,1.387800,1.386670,0.231358,0.156058
3,1.388500,1.386653,0.246654,0.178484


Test Results: {'eval_accuracy': 0.23297262889879058, 'eval_f1': 0.15542993459174664, 'eval_loss': 1.3866671323776245, 'eval_runtime': 43.6874, 'eval_samples_per_second': 35.96, 'eval_steps_per_second': 35.96, 'epoch': 3.0}


## 4.3 Evaluate SQUAD DeBERTa (Acc=26.93%)

In [21]:
path = "./squad-trained-model"
model =  AutoModelForMultipleChoice.from_pretrained(path)
# Create the Trainer
trainer = create_trainer(run_name="Squad-Run")
# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print(f"Test Results: {test_results}")

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at ./squad-trained-model and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Test Results: {'eval_accuracy': 0.2692552514322088, 'eval_f1': 0.2684338987830265, 'eval_loss': 1.3859821557998657, 'eval_model_preparation_time': 0.0, 'eval_runtime': 43.594, 'eval_samples_per_second': 36.037, 'eval_steps_per_second': 36.037}


## 4.4 Evaluate Trained SQUAD DeBERTa (Acc=32.15%)


In [23]:
path = "./squad-trained-model"
model =  AutoModelForMultipleChoice.from_pretrained(path)
# Create the Trainer
trainer = create_trainer(run_name="Squad-Run")

# Train the model
trainer.train()

# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print(f"Test Results: {test_results}")

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at ./squad-trained-model and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.390200,1.386516,0.286170,0.270741
2,1.387900,1.386186,0.287444,0.287612
3,1.364000,1.360636,0.333333,0.332881


Test Results: {'eval_accuracy': 0.3214513049013367, 'eval_f1': 0.32113408409099214, 'eval_loss': 1.357272744178772, 'eval_runtime': 43.6682, 'eval_samples_per_second': 35.976, 'eval_steps_per_second': 35.976, 'epoch': 3.0}


# End of NoteBook